# LangGraph Single Tool Execution

Understanding how a complete tool execution cycle works in LangGraph.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand the complete message flow** for single tool execution from user query to final response
2. **Examine different message types** (HumanMessage, AIMessage with tool_calls, ToolMessage, AIMessage with content)
3. **Trace the execution cycle** through the graph nodes (LLM → Tools → LLM → End)

## 1. Environment Setup

In [ ]:
# Environment setup
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

## 2. Define Tools

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

## 3. Initialize LLM with Tools

In [ ]:
# Initialize LLM with tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

## 4. Build Graph

In [ ]:
# Build graph
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on tool_calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

## 5. Test Case 1: Currency Conversion

Execute a simple currency conversion and trace the message flow.

In [ ]:
# Test Case 1: Currency Conversion
state = {
    "messages": [HumanMessage(content="What is 1000 USD in EUR?")]
}

print("Query: What is 1000 USD in EUR?")
print("=" * 70)

result = app.invoke(state)
print(f"\nTotal messages: {len(result['messages'])}")
print("=" * 70)

### Message Flow Analysis

Let's examine each message in the conversation to understand the complete execution cycle.

In [ ]:
# Examine message flow
print("MESSAGE FLOW ANALYSIS")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    print(f"\n{'─' * 70}")
    print(f"MESSAGE {i}: {type(msg).__name__}")
    print(f"{'─' * 70}")
    
    if isinstance(msg, HumanMessage):
        print(f"  👤 USER INPUT")
        print(f"  Content: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"  🤖 AGENT DECISION: Call tool(s)")
            for tc in msg.tool_calls:
                print(f"    • Tool: {tc['name']}")
                print(f"      Args: {tc['args']}")
        else:
            print(f"  🤖 AGENT RESPONSE: Final answer")
            print(f"  Content: {msg.content}")
            
    elif isinstance(msg, ToolMessage):
        print(f"  🔧 TOOL RESULT")
        print(f"  Content:\n{msg.content}")

### Deep Dive: Individual Messages

Examine the structure of each message type in detail.

In [ ]:
# Deep dive into each message
print("MESSAGE 1: HumanMessage")
print("=" * 70)
print(f"Content: {result['messages'][0].content}")

In [ ]:
dict(result['messages'][0])

In [ ]:
# Message 2: AIMessage with tool_calls
print("MESSAGE 2: AIMessage (Tool Call Request)")
print("=" * 70)
msg2 = result['messages'][1]
print(f"Has tool_calls: {bool(msg2.tool_calls)}")
print(f"\nTool Calls Detail:")
print(msg2.tool_calls)

In [ ]:
dict(result['messages'][1])

In [ ]:
# Message 3: ToolMessage
print("MESSAGE 3: ToolMessage")
print("=" * 70)
msg3 = result['messages'][2]
print(f"Tool Call ID: {msg3.tool_call_id}")
print(f"\nContent:")
print(msg3.content)

In [ ]:
dict(result['messages'][2])

In [ ]:
# Message 4: Final AIMessage
print("MESSAGE 4: AIMessage (Final Response)")
print("=" * 70)
msg4 = result['messages'][3]
print(f"Has tool_calls: {bool(msg4.tool_calls) if hasattr(msg4, 'tool_calls') else False}")
print(f"\nContent:")
print(msg4.content)

In [ ]:
dict(result['messages'][3])

### Conversation Flow Visualization

Visual summary of the complete execution cycle.

In [ ]:
# Visualize conversation flow
print("CONVERSATION FLOW VISUALIZATION")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER:")
        print(f"    \"{msg.content}\"")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT → Calling tool:")
            for tc in msg.tool_calls:
                print(f"    Tool: {tc['name']}")
                print(f"    Args: {tc['args']}")
        else:
            print(f"\n[{i}] 🤖 AGENT → Final response:")
            print(f"    \"{msg.content}\"")
            
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL RESULT:")
        for line in msg.content.split('\n'):
            print(f"    {line}")

print("\n" + "=" * 70)

## 6. Test Case 2: EMI Calculation

Test with a different tool to verify the pattern holds across different tool types.

In [ ]:
# Test Case 2: EMI Calculator
state2 = {
    "messages": [
        HumanMessage(content="Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
    ]
}

print("Query: Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
print("=" * 70)

result2 = app.invoke(state2)
print(f"\nMessage count: {len(result2['messages'])}")

### Tool Selection Verification

Verify that the LLM correctly selected the EMI calculator tool and extracted the appropriate parameters.

In [ ]:
# Verify tool selection
print("Tool Selection Verification:")
print("=" * 70)
tool_call_msg = result2['messages'][1]
if tool_call_msg.tool_calls:
    tc = tool_call_msg.tool_calls[0]
    print(f"✅ LLM selected: {tc['name']}")
    print(f"\nExtracted parameters:")
    for key, value in tc['args'].items():
        print(f"   {key}: {value}")

In [ ]:
# Display conversation flow for EMI test
print("\nCONVERSATION FLOW:")
print("=" * 70)

for i, msg in enumerate(result2["messages"], 1):
    if isinstance(msg, HumanMessage):
        preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
        print(f"\n[{i}] 👤 USER: \"{preview}\"")
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT: Calling {msg.tool_calls[0]['name']}")
        else:
            preview = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
            print(f"\n[{i}] 🤖 AGENT: \"{preview}\"")
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL: Executed emi_calculator")

In [ ]:
# Show final response
print("\nFINAL RESPONSE:")
print("=" * 70)
print(result2["messages"][-1].content)
print("=" * 70)

## Conclusion

In this notebook, you learned:

✅ **Complete message flow** - A single tool execution creates 4 messages: HumanMessage (user query) → AIMessage with tool_calls (LLM decision) → ToolMessage (tool result) → AIMessage with content (final response)

✅ **Different message types** - Each message type serves a specific purpose: HumanMessage carries user input, AIMessage with tool_calls requests tool execution, ToolMessage returns results, AIMessage with content delivers the final answer

✅ **Execution cycle** - The graph flows through nodes: START → LLM (decides to call tool) → Tools (executes tool) → LLM (synthesizes response) → END

### Next Steps

Next, we'll explore **parallel tool execution** where the LLM calls multiple tools simultaneously in a single turn.